Convert data from the BLS one screen [data seach](https://www.bls.gov/cpi/data.htm) to the format for portfolio Visualizer

In [43]:
from pathlib import Path

import pandas as pd

In [44]:
# Initially clean data by removing rows before the headers,
# columns after December, and incomplete rows

original_data_path = Path("cpi_monthly.csv")
transformed_data_path = Path("cpi_rate_monthly_portfolio_visualizer.csv")

In [45]:
original_data = pd.read_csv(original_data_path)
original_data

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,1913,9.800,9.800,9.800,9.800,9.700,9.800,9.900,9.900,10.000,10.000,10.100,10.000
1,1914,10.000,9.900,9.900,9.800,9.900,9.900,10.000,10.200,10.200,10.100,10.200,10.100
2,1915,10.100,10.000,9.900,10.000,10.100,10.100,10.100,10.100,10.100,10.200,10.300,10.300
3,1916,10.400,10.400,10.500,10.600,10.700,10.800,10.800,10.900,11.100,11.300,11.500,11.600
4,1917,11.700,12.000,12.000,12.600,12.800,13.000,12.800,13.000,13.300,13.500,13.500,13.700
...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,2018,247.867,248.991,249.554,250.546,251.588,251.989,252.006,252.146,252.439,252.885,252.038,251.233
106,2019,251.712,252.776,254.202,255.548,256.092,256.143,256.571,256.558,256.759,257.346,257.208,256.974
107,2020,257.971,258.678,258.115,256.389,256.394,257.797,259.101,259.918,260.280,260.388,260.229,260.474
108,2021,261.582,263.014,264.877,267.054,269.195,271.696,273.003,273.567,274.310,276.589,277.948,278.802


In [46]:
# Initialize an empty DataFrame for the transformed data
transformed_data = pd.DataFrame(columns=["Period", "Return"])

for index, row in original_data.iterrows():
    year = int(row["Year"])
    for index, value in enumerate(row.values[1:]):
        month = index + 1
        date_str = f"{month}/{1}/{year}"
        transformed_data = transformed_data.append(
            {"Period": date_str, "Return": value}, ignore_index=True
        )

# export an index version, which is also compatiable with portfolio visualizer
transformed_data.to_csv(Path("cpi_monthly_portfolio_visualizer.csv"), index=False)
transformed_data

/var/folders/y6/h36m53qj57b1mpz1lg9xrcbm0000gn/T/ipykernel_89578/2956076862.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  transformed_data = transformed_data.append({'Period': date_str, 'Return': value}, ignore_index=True)
/var/folders/y6/h36m53qj57b1mpz1lg9xrcbm0000gn/T/ipykernel_89578/2956076862.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  transformed_data = transformed_data.append({'Period': date_str, 'Return': value}, ignore_index=True)
/var/folders/y6/h36m53qj57b1mpz1lg9xrcbm0000gn/T/ipykernel_89578/2956076862.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  transformed_data = transformed_data.append({'Period': date_str, 'Return': value}, ignore_index=True)
/var/folders/y6/h36m53qj57b1mpz1lg9xrcbm0000gn

,Period,Return
0,1/1/1913,9.800
1,2/1/1913,9.800
2,3/1/1913,9.800
3,4/1/1913,9.800
4,5/1/1913,9.700
...,...,...
1315,8/1/2022,296.171
1316,9/1/2022,296.808
1317,10/1/2022,298.012
1318,11/1/2022,297.711


In [47]:
# Convert the values in the return column to rates of change

# Retrive the initial value
initial_value = transformed_data.iloc[0]["Return"]

# Remove the first row, which is the oldest data
transformed_data = transformed_data.iloc[1:]

# Iterate through the rows and calculate the rate of change
for index, row in transformed_data.iterrows():
    value = row["Return"]
    rate_of_change = (value - initial_value) / initial_value
    initial_value = value
    transformed_data.at[index, "Return"] = rate_of_change

transformed_data

,Period,Return
1,2/1/1913,0.000000
2,3/1/1913,0.000000
3,4/1/1913,0.000000
4,5/1/1913,-0.010204
5,6/1/1913,0.010309
...,...,...
1315,8/1/2022,-0.000354
1316,9/1/2022,0.002151
1317,10/1/2022,0.004056
1318,11/1/2022,-0.001010


In [48]:
transformed_data.to_csv(transformed_data_path, index=False)